# Aprendizaje supervisado: Tasa de cancelación de clientes

Tenemos un dataset de clientes del organismo Beta Bank, el cual tiene un problema: sus clientes se estan yendo mes a mes. Necesitamos entrenar un modelo para predecir si los clientes dejarán el banco pronto. Tenemos los datos sobre el comportamiento/características de los clientes y si terminan su contrato o no.

Utilizaremos un par de modelos diferentes (de bosque aleatorio y regresión logística) para esta tarea, tanto con datos con y sin balanceo: esto es, que el número de elementos en cada clase que se desea categorizar se encuentran igual de representadas -por ejemplo, si hay mil de una clase, habría cerca de mil de la(s) otra(s)-.

Observaremos que el balanceo de datos mejora el resultado y que el modelo del bosque aleatorio aunque más complicado, nos da predicciones aceptables.

## Inicialización

Importemos las librerías que necesitaremos en este trabajo.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

### Cargada y sample de datos

Carguemos los datos que tenemos en nuestro dataset y echemos un vistazo rápido a un sample.

In [2]:
# carguemos nuestro dataset y mostremos un poco de información al respecto
df_original = pd.read_csv("/datasets/Churn.csv")

print("Información del Dataframe\n")
print(df_original.info())

print("\nPequeño sample")
display(df_original.sample(n=10, random_state=54321))

Información del Dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
None

Pequeño sample


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
3731,3732,15568573,Graham,554,Germany,Female,51,NaN,105701.91,1,0,1,179797.79,1
2950,2951,15744919,Genovese,734,Spain,Female,37,0.0,152760.24,1,1,1,48990.50,0
69,70,15755648,Pisano,675,France,Female,21,8.0,98373.26,1,1,0,18203.00,0
1851,1852,15633640,Loewenthal,799,France,Female,52,4.0,161209.66,1,1,1,89081.41,0
3627,3628,15609475,Ricci,604,Spain,Female,39,7.0,98544.11,1,1,1,52327.57,0
1311,1312,15750497,Longo,850,France,Female,37,7.0,153147.75,1,1,1,152235.30,0
1335,1336,15576683,Yin,568,Spain,Female,43,9.0,0.00,1,1,0,125870.79,1
905,906,15675964,Chukwukadibia,672,France,Female,45,9.0,0.00,1,1,1,92027.69,1
6075,6076,15781451,Buccho,504,France,Male,42,3.0,134936.97,2,0,0,135178.91,0
378,379,15677371,Ko,629,Spain,Female,30,2.0,34013.63,1,1,0,19570.63,0


Es interesante notar que parece que toda nuestra información esta completa, a exepción de Tenure: veremos que hacer con ella, pero primero eliminaremos las filas que no serán de utilidad para nosotros. `RowNumber`, `Surname`, y `CustomerId` se irán, pero primero checaremos si hay duplicados a partir de CustomerId.

Antes de esto, platiquemos brevemente de lo que tenemos en cada una: `RowNumber` es un entero que da el número de fila (el ID+1 ya que este empieza en cero), `CustomerId` es un identificador entero único por cliente, `Surname` es un string del apellido, `CreditScore` es un entero que indica el rating que tiene cada usuario por su historial de crédito, `Geography` es un string del lugar de residencia, `Gender` es un string que indica el género, `Age` es un entero que marca la edad, `Tenure` es el número de años con el depósio, `Balance` es un float que indica cuanto dinero hay en la cuenta, `NumOfProducts` es un int que marca cuantos productos contratados tiene el cliente, `HasCrCard` es un int que marca si el usuario tiene una tarjeta de crédito, `IsActiveMember` de tipo int indica si tiene tarjeta de crédito, `EstimatedSalary` es un float para el salario estimado del cliente, y finalmente `Exited` es un int para indicar si abandonó el banco.

### Tratamiento de datos
Usaremos duplicated para ver si algún valor se halla duplicado (CustomerId debería ser único por cliente).

In [3]:
df_original[df_original.duplicated(subset="CustomerId")]

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited


Parece que no hay CustomerId duplicados, podemos proceder a remover las columnas que no necesitamos.

In [4]:
df_work = df_original.drop(columns=["RowNumber", "CustomerId", "Surname"])

print("Muestra info")
df_work.sample(n=5, random_state=54321)

Muestra info


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
3731,554,Germany,Female,51,NaN,105701.91,1,0,1,179797.79,1
2950,734,Spain,Female,37,0.0,152760.24,1,1,1,48990.50,0
69,675,France,Female,21,8.0,98373.26,1,1,0,18203.00,0
1851,799,France,Female,52,4.0,161209.66,1,1,1,89081.41,0
3627,604,Spain,Female,39,7.0,98544.11,1,1,1,52327.57,0


Todo se ve mejor, pero todavía tenemos algunos valores nulos en Tenure. Aquí podríamos tomar tres decisiones: 1) considerar que esos valores nulos realmente implican un valor de cero (o sea que no ha madurado el depósito) e imputarlo, 2) eliminar la entrada 3) Imputarlo realizando primero una inspección general (por ejemplo, imputando con la media si los datos se comportan de alguna forma que así pareciera). Procederemos con la segunda porque es más sencilla y solo es ~10% del total.

In [5]:
df_work = df_work.dropna()

# Mostremos info
df_work.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      9091 non-null   int64  
 1   Geography        9091 non-null   object 
 2   Gender           9091 non-null   object 
 3   Age              9091 non-null   int64  
 4   Tenure           9091 non-null   float64
 5   Balance          9091 non-null   float64
 6   NumOfProducts    9091 non-null   int64  
 7   HasCrCard        9091 non-null   int64  
 8   IsActiveMember   9091 non-null   int64  
 9   EstimatedSalary  9091 non-null   float64
 10  Exited           9091 non-null   int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 852.3+ KB


Ya tenemos los datos con los que trabajaremos, solo para continuar, obtengamos cuantos valores tenemos para cada uno de los posibles targets en Exited.

In [6]:
df_work.Exited.value_counts()

0    7237
1    1854
Name: Exited, dtype: int64

Los valores positivos son algo así como ~25% de los negativos, tenemos una relación ~4:1. Esto implica que nuestras clases estan desbalanceadas, no lo corregiremos ahora, sino que haremos pruebas para ver que resultados tenemos. Procederemos con preparar nuestras características.

### Preparación de características

Tenemos variables categóricas y numéricas, tenemos que codificar las categóricas y escalar las numéricas para poder hacer los cálculos posteriores, vayamos a eso.

In [7]:
numeric_columns = ["CreditScore", "Age", "Balance", "EstimatedSalary", "NumOfProducts", "Tenure"]
#numeric_columns = ["CreditScore", "Age", "Balance", "EstimatedSalary", "NumOfProducts", "Tenure"]

category_columns = ["Geography", "Gender"]
#category_columns = ["Geography", "Gender", "HasCrCard", "IsActiveMember"]

# Hagamos el escalado
scaler = StandardScaler()
df_work[numeric_columns] = scaler.fit_transform(df_work[numeric_columns])

# Hagamos el encoding
df_work = pd.get_dummies(df_work, columns=category_columns, drop_first = True)

# Mostremos un sample del resultado
df_work.sample(n=10, random_state=54321)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
8607,-0.505539,-0.089927,-1.035627,-0.077893,0.808655,0,0,0.640917,0,1,0,1
1837,-0.930827,1.615426,0.691748,1.223184,0.808655,1,0,0.572457,1,0,0,1
5550,0.853310,0.478524,0.346273,1.470381,-0.912601,0,1,1.683610,0,0,0,0
8770,-0.370691,1.899651,1.037224,-1.227780,0.808655,1,1,1.134910,0,0,0,1
5221,-0.692251,-0.942603,-0.344677,-1.227780,0.808655,1,0,-0.196355,0,0,1,0
6172,-0.858217,0.383782,0.000798,0.177933,4.251167,1,1,0.121034,1,1,0,1
5960,-0.547030,0.857491,0.346273,0.708528,-0.912601,1,0,-1.206938,1,0,0,1
2015,1.330462,0.099557,-1.035627,-0.016887,-0.912601,1,1,-0.820651,0,1,0,0
9165,0.127207,0.668007,-1.035627,1.046177,-0.912601,1,1,-0.727574,0,1,0,0
5810,-1.615438,0.289040,0.691748,-1.227780,0.808655,0,0,0.524514,0,0,1,0


Se ve todo en órden, tenemos nuestras características mejor acomodadas.

## Modelos sin balancear

Utilizaremos dos diferentes modelos para nuestros datos: de regresión logística y de bosque aleatorio. Antes de trabajar con los modelos, tenemos que dividir nuestros datos en los conjuntos de entrenamiento, validación y prueba con una relación de 60:20:20.

In [8]:
# Separemos en target y feature el dataset
df_features = df_work.drop(["Exited"], axis=1)
df_target = df_work.Exited

# Hagamos el split en nuestros conjuntos de entrenamiento, test y validación. Lo hacemos en dos partes,
# el primero divide al 20% y el segundo al 25% para tomar de ese 80% (0.25*0.8 = 0.2 como se desea).
features_train, features_test, target_train, target_test  = train_test_split(
    df_features, df_target, test_size=0.2, random_state = 54321
)

features_train, features_valid, target_train, target_valid  = train_test_split(
    features_train, target_train, test_size=0.25, random_state = 54321
)

#Hagamos un simple chequeo de los rows para ver que todo anda en órden.
print(
    f"Feat | train: {features_train.shape[0]}, test: {features_test.shape[0]}, valid: {features_valid.shape[0]}"
)
print(
    f"Targ | train: {target_train.shape[0]}, test: {target_test.shape[0]}, valid: {target_valid.shape[0]}"
)

Feat | train: 5454, test: 1819, valid: 1818
Targ | train: 5454, test: 1819, valid: 1818


### Entrenamiento de modelos

Para nuestro modelo logístico no tenemos demasiados hiperpárametros que afinar, por lo que haremos solo el entrenamiento con los valores predefinidos y mostraremos el accuracy.

#### Regresión logística

In [9]:
model_logistic_unbalanced = LogisticRegression(random_state=54321)
model_logistic_unbalanced.fit(features_train, target_train)

target_valid_predicted_logistic_unbalanced = model_logistic_unbalanced.predict(features_valid)

print(
    f"accuracy:\t{accuracy_score(target_valid, target_valid_predicted_logistic_unbalanced)}",
    f"\nf1:\t\t{f1_score(target_valid, target_valid_predicted_logistic_unbalanced)}",
    f"\nroc_auc_score:\t{roc_auc_score(target_valid, model_logistic_unbalanced.predict_proba(features_valid)[:,1])}"
)

accuracy:	0.8157315731573157 
f1:		0.33663366336633666 
roc_auc_score:	0.7981998696755221


No son valores particularmente altos para la validación, prosigamos a realizar el entrenamiento de nuestro modelo de bosque aleatorio.

#### Bosque aleatorio

In [10]:
# Creamos un dataframe para guardar resultados
results_random_trees_unbalanced = pd.DataFrame([], columns=["depth", "estimators", "accuracy"])

for depth in tqdm(range(1, 20+1, 1)):
    for est in range(10, 100+1, 10):
        model_random_trees = RandomForestClassifier(n_estimators = est, max_depth=depth, random_state=54321)
        model_random_trees.fit(features_train, target_train)
        
        prediction_random_trees_unbalanced = model_random_trees.predict(features_valid)
        
        results_random_trees_unbalanced = results_random_trees_unbalanced.append(
            {
                "depth": depth,
                "estimators": est,
                "accuracy": accuracy_score(target_valid, prediction_random_trees_unbalanced),
                "f1": f1_score(target_valid, prediction_random_trees_unbalanced),
                "roc_auc_score": roc_auc_score(target_valid, model_random_trees.predict_proba(features_valid)[:,1])
            },
            ignore_index=True
        )

  0%|          | 0/20 [00:00<?, ?it/s]

Veamos cuales son los hiperpárametros con los mejores valores para la métrica f1.

In [11]:
# Hacemos un pequeño sort_values y el head
results_random_trees_unbalanced.sort_values("f1", ascending=False).head(n=5)

,depth,estimators,accuracy,f1,roc_auc_score
188,19.0,90.0,0.871837,0.604414,0.854975
189,19.0,100.0,0.870737,0.602369,0.854152
117,12.0,80.0,0.870187,0.597270,0.855340
179,18.0,100.0,0.866887,0.596667,0.855778
176,18.0,70.0,0.866887,0.596667,0.851877


Nuestros dos modelos aunque básicos, tienen una accuracy notable, pero el valor F1 sigue siendo bajo (y bajaría más al aplicarse al dataset de prueba): este valor de _accuracy_ indica que de todos los valores, se pueden clasificar correctamente ~87% de las entradas; un F1 ~0.60 indica que no se tiene la misma precisión al clasificar las clases positivas (en este caso al estar en una clasificación binaria); finalmente, un área bajo la curva ROC ~0.85 indica que se puede distinguir decentemente Verdaderos Positivos y Falsos Positivos. Todo suena muy bien, pero tener un F1 tan bajo no es buena señal ya que la precisión y la sensibilidad no se hallan altas ambas, busquemos una manera de mejorarlo.

## Modelados con balanceo de clases

Uno de los problemas de nuestros datos es que nuestras clases están desbalanceadas: tenemos 4 veces más negativos que positivos en nuestra característica de salida, lo cual afecta nuestros modelos; tenemos que balancear los valores para que sean más similares. Primero veamos como es la relación de positivos a negativos en el dataset de entrenamiento.

In [12]:
target_train.value_counts()

0    4355
1    1099
Name: Exited, dtype: int64

Notemos que tenemos una relación similar a la de nuestro dataset original. Podemos proceder haciendo un sobre/submuestreo o cambiando el peso de clase, haremos sobremuestreo y luego cambiaremos el peso; como nuestra relación es de cuatro a uno apróximadamente, repetiremos los datos cuatro veces para que sean similares en número.

### Modelado con sobremuestreo

In [13]:
# Usemos otra vez df_worked ya que tiene la información codificada y escalada.
# Hagamos el upsample de las muestras: no necesitamos una función ya que solo lo haremos una vez

# Haremos la repetición 4 veces 
repeat = 4
    
# Separamos los registros
features_zeros = features_train[target_train == 0]
features_ones = features_train[target_train == 1]
target_zeros = target_train[target_train == 0]
target_ones = target_train[target_train == 1]


features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
features_upsampled, target_upsampled = shuffle(
    features_upsampled, target_upsampled, random_state=54321
)

# Mostremos lo que tenemos
target_upsampled.value_counts()


1    4396
0    4355
Name: Exited, dtype: int64

Ya se encuentran muy similares, podemos proceder con el entrenamiento.

#### Regresión logística

In [14]:
model_logistic_balanced = LogisticRegression(random_state=54321)
model_logistic_balanced.fit(features_upsampled, target_upsampled)

target_valid_predicted_logistic_balanced = model_logistic_balanced.predict(features_valid)

print(
    f"accuracy:\t{accuracy_score(target_valid, target_valid_predicted_logistic_balanced)}",
    f"\nf1:\t\t{f1_score(target_valid, target_valid_predicted_logistic_balanced)}",
    f"\nroc_auc_score:\t{roc_auc_score(target_valid, model_logistic_balanced.predict_proba(features_valid)[:,1])}"
)

accuracy:	0.724972497249725 
f1:		0.5274102079395085 
roc_auc_score:	0.8008138707308874


Note que el modelo de regresión logística empeora en accuracy (en comparación con los datos sin balancear), pero las demás métricas mejoran sustancialmente.

#### Bosque aleatorio

In [15]:
# Creamos un dataframe para guardar resultados
results_random_trees_balanced = pd.DataFrame([])

for depth in tqdm(range(1, 20+1, 1)):
    for est in range(10, 100+1, 10):
        model_random_trees = RandomForestClassifier(n_estimators = est, max_depth=depth, random_state=54321)
        model_random_trees.fit(features_upsampled, target_upsampled)
        
        predictions_random_trees_balanced = model_random_trees.predict(features_valid)
        
        results_random_trees_balanced = results_random_trees_balanced.append(
            {
                "depth": depth,
                "estimators": est,
                "accuracy": accuracy_score(target_valid, predictions_random_trees_balanced),
                "f1": f1_score(target_valid, predictions_random_trees_balanced),
                "roc_auc_score": roc_auc_score(target_valid, model_random_trees.predict_proba(features_valid)[:,1])
            },
            ignore_index=True

        )

  0%|          | 0/20 [00:00<?, ?it/s]

In [16]:
results_random_trees_balanced.sort_values("f1", ascending=False).head()

,accuracy,depth,estimators,f1,roc_auc_score
114,0.859186,12.0,50.0,0.654054,0.864333
117,0.859186,12.0,80.0,0.651226,0.867167
106,0.851485,11.0,70.0,0.651163,0.871987
116,0.859736,12.0,70.0,0.651163,0.868057
105,0.851485,11.0,60.0,0.649351,0.870932


De la tabla de resultados podemos decantarnos por depth=12 y 50 estimadores como hiperpárametros. Procedamos a ver que sucede cuando realizamos los procedimientos pero utilizando el balanceo de clases en el argumento de los modelos.

### Modelado con peso ajustado

Realicemos el mismo procedimiento de la sección anterior, pero utilizando el argumento `class_weight='balanced'` para obtener el balanceo automáticamente sin tener que realizar sobre/infra muestreo.

#### Regresión logística

Primero, el modelo de regresión logística.

In [17]:
model_logistic_weighted = LogisticRegression(random_state=54321, class_weight='balanced')
model_logistic_weighted.fit(features_train, target_train)

target_valid_predicted_logistic_weighted = model_logistic_weighted.predict(features_valid)

print(
    f"accuracy:\t{accuracy_score(target_valid, target_valid_predicted_logistic_weighted)}",
    f"\nf1:\t\t{f1_score(target_valid, target_valid_predicted_logistic_weighted)}",
    f"\nroc_auc_score:\t{roc_auc_score(target_valid, model_logistic_weighted.predict_proba(features_valid)[:,1])}"
)

accuracy:	0.7266226622662266 
f1:		0.5280151946818614 
roc_auc_score:	0.8008026034849591


Que interesante, las métricas se mantienen muy similares a las del sobremuestreo a pesar de ser mucho más sencillo de operar.

#### Bosque Aleatorio

Prosigamos con el segundo modelo, de Bosque Aleatorio:

In [18]:
# Creamos un dataframe para guardar resultados
results_random_trees_weighted = pd.DataFrame([], columns=["depth", "estimators", "accuracy"])

for depth in tqdm(range(1, 20+1, 1)):
    for est in range(10, 100+1, 10):
        model_random_trees = RandomForestClassifier(
            n_estimators = est, max_depth=depth, random_state=54321, class_weight='balanced'
        )
        model_random_trees.fit(features_train, target_train)
        
        prediction_random_trees_weighted = model_random_trees.predict(features_valid)
        
        results_random_trees_weighted = results_random_trees_weighted.append(
            {
                "depth": depth,
                "estimators": est,
                "accuracy": accuracy_score(target_valid, prediction_random_trees_weighted),
                "f1": f1_score(target_valid, prediction_random_trees_weighted),
                "roc_auc_score": roc_auc_score(target_valid, model_random_trees.predict_proba(features_valid)[:,1])
            },
            ignore_index=True
        )

  0%|          | 0/20 [00:00<?, ?it/s]

In [19]:
results_random_trees_weighted.sort_values("f1", ascending=False).head()

,depth,estimators,accuracy,f1,roc_auc_score
82,9.0,30.0,0.852585,0.641711,0.869901
83,9.0,40.0,0.850935,0.639148,0.872288
86,9.0,70.0,0.852585,0.638814,0.872218
85,9.0,60.0,0.852585,0.638814,0.871536
87,9.0,80.0,0.854785,0.638356,0.873972


Parece que el modelo de bosque aleatorio con el peso ajustado a las clases da el mejor resultado de F1. Podríamos ir directamente con profundidad 9 y 30 estimadores, pero nótese que el área bajo la curva ROC disminuye un poco justo en estos hiperpárametros: empeora en su capacidad de distinguir las clases. Queremos optimizar F1, pero no por eso tomaremos cualquiera hiperpárametros, viendo de la tabla el valor de 80 estimadores, se tiene un ROC-AUC más alto y el F1 solo disminuye unas centésimas: este es el que elegiremos.

## Métricas para el conjunto de prueba

Ya tenemos nuestro modelo elegido y con los hiperpárametros apropiados, utilizaremos el dataset de prueba para poder obtener las métricas finales que buscamos.

In [24]:
# Entrenemos el modelo con estos hiperpárametros
model_final_random_trees = RandomForestClassifier(
    n_estimators=80, max_depth=9, random_state=54321, class_weight='balanced'
)

# Usaremos tanto los datos de training como de validación para mejorar resultados
model_final_random_trees.fit(
    pd.concat([features_train] + [features_valid]), pd.concat([target_train] + [target_valid])
)

prediction_final_model = model_final_random_trees.predict(features_test)


# Mostremos las métricas para el conjunto de prueba
print(
    f"accuracy:\t{accuracy_score(target_test, prediction_final_model)}",
    f"\nf1:\t\t{f1_score(target_test, prediction_final_model)}",
    f"\nroc_auc_score:\t{roc_auc_score(target_test, model_final_random_trees.predict_proba(features_test)[:,1])}"
)

accuracy:	0.817482133040132 
f1:		0.5951219512195122 
roc_auc_score:	0.8478588976060285


Hemos logrado obtener un modelo que nos devuelve una accuracy mayor al 80%, lo que indica que tiene buena capacidad de predecir el target deseado (con reservas, ya que tenemos clases desbalanceadas). Por otro lado, el valor de F1 a 0.59 indica que tenemos un modelo que es moderado-fuerte para distinguir los valores en sus clases; el área bajo la curva de ~0.85 igual indica que tenemos una buena distinguibilidad. Solo para poder comparar mejor, realicemos estos cálculos con el modelo de árboles aleatorios sin balancear (profundidad máxima de 19, y 90 estimadores).

In [25]:
# Entrenemos el modelo con estos hiperpárametros
model_final_unbalanced_random_trees = RandomForestClassifier(
    n_estimators=90, max_depth=19, random_state=54321
)

# Usaremos tanto los datos de training como de validación para mejorar resultados
model_final_unbalanced_random_trees.fit(
    pd.concat([features_train] + [features_valid]), pd.concat([target_train] + [target_valid])
)

prediction_final_model_unbalanced = model_final_unbalanced_random_trees.predict(features_test)

# Mostremos las métricas para el conjunto de prueba
print(
    f"accuracy:\t{accuracy_score(target_test, prediction_final_model_unbalanced)}",
    f"\nf1:\t\t{f1_score(target_test, prediction_final_model_unbalanced)}",
    f"\nroc_auc_score:\t{roc_auc_score(target_test, model_final_unbalanced_random_trees.predict_proba(features_test)[:,1])}"
)        

accuracy:	0.8543155579989005 
f1:		0.573268921095008 
roc_auc_score:	0.8500453867600337


Estos valores son muy similares a los que tenemos para el modelo con clases balanceadas, incluso _accuracy_ es mayor para este, pero el valor F1 es menor: la diferencia es de apenás unas décimas, pero un cambio para esta métrica puede ser la diferencia para poder predecir valores realmente positivos de falsos positivos.

## Conclusiones

Hemos trabajado con los datos proporcionados por el banco: hicimos un análisis rápido de nuestra información y preparamos nuestros datos haciendo codificación y escalado de características. Procedemos a entrenar los dos modelos sin y con balanceo de clases, dandonos cuenta de la mejoría del segundo. Con los hiperpárametros obtenidos para el modelo de Bosque aleatorio implementamos para el conjunto de prueba y obtenemos las métricas que estamos buscando.

Hay una diferencia pequeña, pero notable cuando el dataset esta y no balanceado: cuando tenemos una aplicación crítica siempre hay que buscar la mayor precisión para poder obtener resultados más fiables. El accuracy es una métrica muy útil, pero aquí nos da impresión de ser más importante de lo que es -al tener clases sin balancear-, así que nos debemos de fiar más de de F1 y del área bajo la curva ROC para tener un buen modelo.